# TransferMarket - Most valueable players 데이터 크롤링하기
* [TransferMarket](https://www.transfermarkt.com/)은 축구 선수, 클럽 등의 기록, 정보, 이적 기록 등을 모아놓은 해외 사이트이다.
* 그 중 Most valueable players 탭을 보면, 가장 몸값이 비싼 축구선수 top500위가 정리되어있다.
* 500명의 선수들에 대한 다양한 정보들을 직접 크롤링하는 것이 목표이다.

In [106]:
import pandas as pd
from selenium import webdriver
import time

In [109]:
# webdriver로 자동화된 크롬브라우저 실행
driver = webdriver.Chrome('D:\작업\chromedriver\chromedriver')
driver.get('https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop')

In [ ]:
# 선수 정보를 가져와서 저장할 딕셔너리 생성
player_info = {
    'name':[],
    'age':[],
    'height': [],
    'country': [],
    'position': [],
    'foot': [],
    'team': [],
    'joined': [],    # 현재 팀 입단 일자
    'contract expires': [],    # 계약 만료일
    'goals': [],
    'assists': [],
    'appearances': [],
    'minutes played': [],
    'market value': []
}

### 크롤러 코드
* 마주쳤던 문제점 1) 각 선수마다 이름의 xpath가 다른 경우가 많았다.
    * 예를 들어 어떤 선수는 xpath의 위치가 ...tr:nth-child(1) 인데, 어떤 선수는 ...tr:nth-child(2)인 경우가 있었다.
    * 이를 파악하기 위해 직접 살펴본 결과, 등록된 이름이 fullname이 아니거나, 아랍어 등 타언어일 경우는 Name in home country, fullname 같은 항목이 추가로 기재되어있었다.
        * 예를 들면, Dele Alli의 경우, Name in home country이 Bamidele Jermaine Alli라고 기재되어있다.
        * 반면, Sadio Mané는 그 이름 자체가 fullname이므로, fullname이나 Name ... country 라는 항목 자체가 없다.
    * **문제 해결**: if문으로 두 case로 나누어 크롤러를 만들었다.

* 마주쳤던 문제점 2) 선수의 정보를 수집 후, 다음 선수의 정보를 수집할 때 뒤로 돌아가면 무조건 1페이지가 나왔다.
    * 가장 난관이었다...
    * 이를 해결하기 위해 크롤링을 하기 전, n페이지까지 도달하기 위해 n-1번의 '다음 페이지'버튼을 누르는 코드를 넣어야 했다.
    * **문제 해결**: for문을 통해, 2번째 페이지부터는 다음 페이지 버튼을 누르도록 코드를 짰다.
    
* 마주쳤던 문제점 3) ip차단
    * 보안상의 문제로 대다수의 도메인들은 사람인지, 기계인지 우리를 지켜보고있다.
    * 이 문제를 해결하기 위해서는 한 동작이 이뤄질 때 마다 약간의 시간 간극을 줘야 한다.
    * 몇 초를 줘야하는지는 각 도메인마다 다른 것으로 알고있다.
    * **문제 해결**: 경험적으로 3~5초를 지정했다.

* 마주쳤던 문제점 4) 웹을 최대화하지 않았더니 error 발생
    * 자동화된 웹의 크기를 조금 작게 해놨더니, 브라우저에서 보이지 않는 영역은 크롤링이 되지 않았다.
        * 예를들어, 선수가 1\~25번까지 있는데 창을 작게해놔서 1~10번까지의 선수는 보이지 않는다.
        * 그러면 그 선수들은 크롤링이 되지 않고 error로 처리되었다.
    * **문제 해결**: 창을 최대화해서 모든 선수가 보이도록 한다. 
    * *11페이지까지 도달했을 때 이 문제를 발견해서, 많은 수의 데이터를 수기로 입력해야한다.*

* 해결한 줄 알았는데 알고보니 해결이 안 된 문제
    * 문제 2)번에서 다음 페이지 버튼을 클릭하는 코드에서 문제가 발생했다.
        * 코드 자체는 잘 작동했지만, 10페이지정도가 넘어간 순간부터는 버튼을 눌렀을 때 웹이 느리게 동작했다.
        * 그래서 13페이지까지 가야하는데 정작 웹은 10페이지까지밖에 도달하지 못하는 등의 문제가 발생했다.
    * 더 문제는, 이걸 19페이지까지 돌아갔을 때 알아차렸다는 것!

In [63]:
# 트랜스퍼마켓 접속
driver.get('https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop')
driver.implicitly_wait(10)
time.sleep(3)
for pagenum in range(0, 20):
    # 첫 페이지인 경우, '1페이지 시작!' 출력
    if pagenum == 0:
        print('1페이지 시작!')    
    # 첫 페이지가 아닌 경우, 몇 번째 페이지인지 출력
    # 그리고 다음 페이지 버튼을 n번 클릭
    else:
        print('{}번째 페이지를 시작합니다.'.format(pagenum+1))
        time.sleep(5)
        driver.find_element_by_xpath('//*[@id="yw2"]/li[13]/a').click()
        driver.implicitly_wait(10)
        time.sleep(5)
    
    # 선수의 세부 정보를 확인할 수 있도록 click
    # 이후 각 요소들을 크롤링함
    for num in range(1, 26):
        driver.implicitly_wait(10)
        try:
            time.sleep(5)
            driver.find_element_by_css_selector('#yw1 > table > tbody > tr:nth-child({}) > td:nth-child(2) > table > tbody > tr:nth-child(1)'.format(num)).click()
            driver.implicitly_wait(10)
            time.sleep(5)
            # 이름 xpath가 달라서 생기는 오류 해결
            try:
                player_info['name'].append(driver.find_element_by_xpath('//*[@id="main"]/div[8]/div/div/div[1]/div/div[1]/h1').text)
            except:
                player_info['name'].append(driver.find_element_by_xpath('//*[@id="main"]/div[8]/div/div/div[2]/div/div[1]/h1').text)
            # fullname이 있는 경우와 없는 경우 나누기
            if driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[1]/th').text == 'Name in home country:' or driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[1]/th').text =='Full name:':
                player_info['age'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[4]/td').text)
                player_info['height'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[5]/td').text)
                player_info['country'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[6]/td').text)
                player_info['position'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[7]/td').text)
                player_info['foot'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[8]/td').text)
                player_info['team'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[10]/td').text)
                player_info['joined'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[11]/td').text)
                player_info['contract expires'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[12]/td').text)
            else:
                player_info['age'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[3]/td').text)
                player_info['height'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[4]/td').text)
                player_info['country'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[5]/td').text)
                player_info['position'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[6]/td').text)
                player_info['foot'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[7]/td').text)
                player_info['team'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[9]/td').text)
                player_info['joined'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[10]/td').text)
                player_info['contract expires'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[11]/td').text)
            player_info['goals'].append(driver.find_element_by_xpath('//*[@id="yw2"]/table/tfoot/tr/td[4]').text)
            player_info['assists'].append(driver.find_element_by_xpath('//*[@id="yw2"]/table/tfoot/tr/td[5]').text)
            player_info['appearances'].append(driver.find_element_by_xpath('//*[@id="yw2"]/table/tfoot/tr/td[3]').text)
            driver.implicitly_wait(10)
            player_info['minutes played'].append(driver.find_element_by_xpath('//*[@id="yw2"]/table/tfoot/tr/td[7]').text)
            player_info['market value'].append(driver.find_element_by_xpath('//*[@id="main"]/div[10]/div[1]/div[2]/div[2]/div[1]/div/div[4]/div[1]/div[1]/div/div[1]/div[2]').text)
            
            # 크롤링한 선수의 이름 출력
            try:
                print(driver.find_element_by_xpath('//*[@id="main"]/div[8]/div/div/div[1]/div/div[1]/h1').text)
            except:
                print(driver.find_element_by_xpath('//*[@id="main"]/div[8]/div/div/div[2]/div/div[1]/h1').text)
            driver.implicitly_wait(10)
            time.sleep(5)
            # 크롤링을 모두 완료했으니, 다시 선수 리스트 화면으로 돌아가기
            driver.back()
            driver.implicitly_wait(10)
            
            # 근데 뒤로가기를 누르면 1페이지로 돌아간다... (후...)
            # n페이지까지 도달하기 위해 n-1번 다음 페이지 버튼을 클릭한다.
            for pn in range(0, pagenum):
                driver.find_element_by_xpath('//*[@id="yw2"]/li[13]/a').click()
                driver.implicitly_wait(10)
                time.sleep(5)
        
        # 오류가 발생했을 때에는 error로 모든 행에 기록하고, error를 출력한다.
        # 그 후 다시 n페이지까지 돌아간다.
        except:
            for n in a.keys():
                a[n].append('error')
            print('error!')
            driver.get('https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop')
            for pn in range(0, pagenum):
                driver.find_element_by_xpath('//*[@id="yw2"]/li[13]/a').click()
                driver.implicitly_wait(10)
                time.sleep(3)

1페이지 시작!
error!
Kylian Mbappé
Raheem Sterling
Neymar
Sadio Mané
Mohamed Salah
Harry Kane
Kevin De Bruyne
Lionel Messi
Jadon Sancho
Antoine Griezmann
Trent Alexander-Arnold
Bernardo Silva
N'Golo Kanté
Leroy Sané
Virgil van Dijk
Paul Pogba
error!
Eden Hazard
João Félix
Frenkie de Jong
Kai Havertz
Paulo Dybala
Serge Gnabry
Saúl Ñíguez
Roberto Firmino
2번째 페이지를 시작합니다.
error!
error!
error!
error!
error!
error!
error!
error!
Sergej Milinkovic-Savic
Marcus Rashford
Andrew Robertson
Dele Alli
Timo Werner
Raphaël Varane
Joshua Kimmich
Heung-min Son
Casemiro
Matthijs de Ligt
Aymeric Laporte
Marco Verratti
Mauro Icardi
Cristiano Ronaldo
Gabriel Jesus
Arthur
Ousmane Dembélé
Lucas Hernández
3번째 페이지를 시작합니다.
error!
error!
error!
error!
error!
error!
error!
error!
error!
Tanguy Ndombélé
Richarlison
Nicolas Pépé
Marquinhos
Jorginho
David Alaba
Miralem Pjanic
Sergio Agüero
Achraf Hakimi
Federico Valverde
Mikel Oyarzabal
Fabián Ruiz
Davinson Sánchez
Federico Chiesa
Christian Pulisic
James Maddison
Bruno F

KeyboardInterrupt: 

In [94]:
df = pd.DataFrame(player_info)
df

,name,age,height,country,position,foot,team,joined,contract expires,goals,assists,appearances,minutes played,market value
0,error,error,error,error,error,error,error,error,error,error,error,error,error,error
1,Kylian Mbappé,21,"1,78 m",France,Forward - Centre-Forward,right,Paris Saint-Germain,"Jul 1, 2018",30.06.2022,30,17,33,2.479,€200.00m
2,Raheem Sterling,25,"1,70 m",England\n Jamaica,Forward - Left Winger,right,Manchester City,"Jul 14, 2015",30.06.2023,20,7,39,3.002,€160.00m
3,Neymar,28,"1,75 m",Brazil,Forward - Left Winger,right,Paris Saint-Germain,"Aug 3, 2017",30.06.2022,18,10,22,1.906,€160.00m
4,Sadio Mané,27,"1,74 m",Senegal,Forward - Left Winger,right,Liverpool FC,"Jul 1, 2016",30.06.2023,18,12,38,3.049,€150.00m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,Dodi Lukebakio,22,"1,87 m",Belgium\n DR Congo,Forward - Right Winger,left,"Aug 1, 2019",30.06.2024,error,error,error,error,error,error
501,error,error,error,error,error,error,error,error,error,error,error,error,error,error
502,error,error,error,error,error,error,error,error,error,error,error,error,error,error
503,error,error,error,error,error,error,error,error,error,error,error,error,error,error


In [91]:
# 레코드 수 확인해보기
for name in player_info.keys():
    print('{}의 레코드 수: {}'.format(name, len(player_info[name])))

name의 레코드 수: 505
age의 레코드 수: 505
height의 레코드 수: 505
country의 레코드 수: 505
position의 레코드 수: 505
foot의 레코드 수: 505
team의 레코드 수: 505
joined의 레코드 수: 505
contract expires의 레코드 수: 503
goals의 레코드 수: 503
assists의 레코드 수: 505
appearances의 레코드 수: 505
minutes played의 레코드 수: 505
market value의 레코드 수: 505
